# Optimisation Test Notebook - Insertion Heuristic

This notebook is created so the dynamic optimisation using the insertion heuristic can be performed.

In [1]:
import os

import numpy as np
import pandas as pd
import plotly.graph_objects as go

### 1. Set up of the model

MODIFY

In [2]:
import Main_insertion

In [3]:
# get the file names of the excel files in Results_requests folder
files = os.listdir('../Results_requests')
# only the requests files
files = [file for file in files if 'Requests_' in file]

# let the user choose the file
# file can be distiguished by vessel type, demand level, and the instance number
# the user can choose the file by the vessel type, demand level, and the instance number

vessel_types = {'Parcel': 0 , 'Passenger': 1, 'Mixed': 2}
demand_levels = ['low', 'high']
# depends on the max value of the instance number
N_inst = len(files) / (2*7)

# let the user choose - type your input
vessel_type_0 = input('Enter the vessel type for vessel 0 (Parcel, Passenger, Mixed): ')
vessel_type_0 = vessel_types[vessel_type_0]
vessel_type_1 = input('Enter the vessel type for vessel 1 (Parcel, Passenger, Mixed): ')
vessel_type_1 = vessel_types[vessel_type_1]
demand_levels = input('Enter the demand level (low, high): ')
instance_number = input(f'Enter the instance number (0-{int(N_inst-1)}): ')

# get the file name
filename = f'Requests_K0_{vessel_type_0}_K1_{vessel_type_1}_demand_{demand_levels}_instance_{instance_number}.xlsx'

In [4]:
data_path = f'../Results_requests/{filename}'

# output folder
if not os.path.exists('../Results_insertion'):
    os.makedirs('../Results_insertion')

output_folder = '../Results_insertion/'

In [5]:
df_result = Main_insertion.main(data_path, output_folder, scenario=demand_levels, instance=instance_number, pattern=[vessel_type_0, vessel_type_1])

Time step 0
Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[4], [6, 4, 7]]
New objective value
3.75484636363642
Time step 3
Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9], [10, 0, 2, 4, 5, 7]]
New objective value
5.8385465281361695
Time step 18
Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9], [10, 2, 1, 4, 5, 7]]
New objective value
4.049729463488359
Time step 25
Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9], [10, 2, 5, 1, 0, 4, 5, 7]]
New objective value
6.7869578521668785
Time step 26
Insertion done
Solving time: 0.001 seconds
New sequences and routes
[[9], [10, 2, 5, 6, 1, 0, 4, 5, 7]]
New objective value
8.92715820201007
Time step 27
Insertion done
Solving time: 0.001 seconds
New sequences and routes
[[9], [10, 2, 5, 6, 1, 0, 4, 5, 7]]
New objective value
8.92715820201007
Time step 32
Insertion done
Solving time: 0.0005 seconds
New sequences and routes
[[9], [10, 2, 5, 6, 7, 1, 0, 4, 5, 7]]


c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  R.loc[r, 'st'] = time
c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\Main_insertion.py:209: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_distance = df_distance._append(row, ignore_index=True)
c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning

Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9, 4], [10, 7, 5, 2, 1, 0, 4, 5, 7]]
New objective value
8.276254696730465
Time step 67
Insertion done
Solving time: 0.0037 seconds
New sequences and routes
[[9], [10, 5, 4, 2, 1, 0, 4, 5, 7]]
New objective value
7.24418947837504
Time step 69
Insertion done
Solving time: 0.0046 seconds
New sequences and routes
[[9], [10, 7, 5, 4, 3, 2, 1, 0, 4, 5, 7]]
New objective value
8.601634463410736
Time step 80
Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9], [10, 5, 4, 3, 2, 1, 0, 4, 5, 7, 4]]
New objective value
9.254967464712035
Time step 84
Insertion done
Solving time: 0.0046 seconds
New sequences and routes
[[9], [10, 5, 4, 3, 2, 1, 0, 4, 5, 6, 7, 4]]
New objective value
8.712181845952127
Time step 107
Insertion done
Solving time: 0.001 seconds
New sequences and routes
[[9], [10, 3, 2, 1, 0, 4, 5, 6, 7, 4]]
New objective value
6.9693417821722
Time step 108


c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  R.loc[r, 'st'] = time
c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  R.loc[r, 'st'] = time
c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Valu

Insertion done
Solving time: 0.167 seconds
New sequences and routes
[[9, 2, 7], [10, 3, 2, 1, 0, 4, 5, 6, 7, 4]]
New objective value
11.249749071020322
Time step 111
Insertion done
Solving time: 0.001 seconds
New sequences and routes
[[9, 2, 7], [10, 3, 2, 1, 0, 4, 5, 6, 7, 4]]
New objective value
10.503627599729862
Time step 115
Insertion done
Solving time: 0.0029 seconds
New sequences and routes
[[9, 2, 7], [10, 3, 2, 1, 0, 4, 5, 6, 7, 4]]
New objective value
9.882854754846461
Time step 118
Insertion done
Solving time: 0.001 seconds
New sequences and routes
[[9, 7], [10, 3, 2, 1, 0, 4, 5, 6, 7, 4]]
New objective value
9.603041778138431
Time step 119
Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9, 7], [10, 3, 2, 1, 0, 4, 5, 6, 7, 4]]
New objective value
9.416500405711512
Time step 124
Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9, 7], [10, 2, 1, 0, 4, 5, 6, 7, 4, 1]]
New objective value
9.784889613323974
Time step 128
Insertion done
Solvi

...

c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  R.loc[r, 'st'] = time
c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  R.loc[r, 'st'] = time
c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Valu

Insertion done
Solving time: 0.0013 seconds
New sequences and routes
[[9, 1, 3, 4], [10, 7, 3, 1, 2, 0]]
New objective value
6.861880597474825
Time step 785
Insertion done
Solving time: 0.0094 seconds
New sequences and routes
[[9, 1, 3, 4], [10, 7, 3, 1, 2, 0]]
New objective value
6.115769411355372
Time step 790
Insertion done
Solving time: 0.0082 seconds
New sequences and routes
[[9, 1, 3, 4], [10, 3, 1, 2, 0]]
New objective value
4.820531197247193
Time step 793
Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9, 1, 3, 4], [10, 3, 1, 2, 0]]
New objective value
4.260937235786112
Time step 806
Insertion done
Solving time: 0.0122 seconds
New sequences and routes
[[9], [10, 1, 2, 0]]
New objective value
1.2320002492372915
Time step 828
Insertion done
Solving time: 0.001 seconds
New sequences and routes
[[9, 4, 2], [10]]
New objective value
1.1803619628856208
Time step 841
Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9], [10, 1, 4]]
New objective v

c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  R.loc[r, 'st'] = time
c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  R.loc[r, 'st'] = time
c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Valu

Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9], [10, 2, 5, 6, 7]]
New objective value
4.3354174362474405
Time step 866
Insertion done
Solving time: 0.0014 seconds
New sequences and routes
[[9], [10, 2, 3, 4, 5, 6, 7]]
New objective value
3.9631240813596786
Time step 870
Insertion done
Solving time: 0.0007 seconds
New sequences and routes
[[9], [10, 2, 1, 3, 4, 5, 6, 7]]
New objective value
4.581922051835034
Time step 873
Insertion done
Solving time: 0.0002 seconds
New sequences and routes
[[9], [10, 1, 3, 4, 5, 6, 7]]
New objective value
4.4060355242010205
Time step 876
Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9], [10, 0, 3, 4, 5, 6, 7]]
New objective value
4.445092229894879
Time step 884
Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9], [10, 4, 5, 6, 7]]
New objective value
3.4191577717614807
Time step 892
Insertion done
Solving time: 0.0 seconds
New sequences and routes
[[9, 0, 6], [10, 5, 6, 7]]
New objective v

c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  R.loc[r, 'st'] = time
c:\Users\heisu\OneDrive\ドキュメント\TU Delft\Thesis\repo\DynamicFleetManagementForMixedService-1\Src\Optimisation\insertion.py:1181: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  R.loc[r, 'st'] = time


In [9]:
df_result.tail()

,Time,Total_dist,Total_empty_dist,Travel_distance_k0,Travel_distance_k1,Empty_distance_k0,Empty_distance_k1,loading_pass_0,loading_pass_1,loading_parcel_0,loading_parcel_1,obj,Met_Ratio,N_requests,Duration
170,884.0,145.366546,11.068792,75.806733,69.559814,8.117355,2.951437,0.0,2.0,0.0,17.0,3.419158,0.845714,8.0,0.0
171,892.0,146.461347,11.068792,75.806733,70.654615,8.117355,2.951437,0.0,0.0,0.0,16.0,4.945712,0.848315,6.0,0.0
172,893.0,146.719033,11.233209,75.971150,70.747882,8.281772,2.951437,0.0,0.0,0.0,17.0,7.185295,0.848315,7.0,0.0
173,912.0,151.188524,11.233209,78.209553,72.978971,8.281772,2.951437,0.0,0.0,8.0,0.0,5.847570,0.852459,3.0,0.0
174,948.0,157.036094,11.233209,84.057122,72.978971,8.281772,2.951437,0.0,0.0,0.0,0.0,4.241989,0.854839,1.0,0.0
